In [1]:
import pandas as pd
import os 
import cv2 as cv

In [2]:
files = os.listdir('../wormvids/')

FileNotFoundError: [Errno 2] No such file or directory: '../wormvids/'

In [ ]:
files.remove('.DS_Store')

This is a big block of code which goes through and openes each file and records the data. The comments are located in the code

In [ ]:
for fil in files:
    all_centers = []
    path = '../wormvids/'+fil
    print(path)
    cap = cv.VideoCapture(path)
    
    if (cap.isOpened() == False):
        print(f'Error opening {path}')
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if (ret == True):
            capbw = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) # Converts the video to black and white

            ret, thresh = cv.threshold(capbw, 127, 255, 0) # Makes a threshold for contours.
            _, contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) # Finds all contours based off the threshold above

            areas = []
            centers = []

            for contour in contours: # Goes through each contour and removes all with area less than 1000 and greater than 5000
                if (cv.contourArea(contour) >  1000) & (cv.contourArea(contour) < 5000):
                    areas.append(contour)
                    M = cv.moments(contour) # This makes a center dot based off the contour
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    centers.append((cX, cY))
            all_centers.append(centers)

        elif ret == True: # Skips if count is not mod 2.
            pass
        else: # Breaks if there is an error
            break
    
    
    df = pd.DataFrame(all_centers)
    df.to_csv('../worm_centers_csv/' + fil.strip('.avi') + '.csv', index = False)